In [ ]:
import os
import whisper
from pyannote.audio import Pipeline
import datetime
import itertools
import numpy as np
from pydub.playback import play
from pydub import AudioSegment
import pandas as pd
import torch
import yaml

class TranscriptLine:
    def __init__(self, start = None, end = None, speaker_dict = {}, speaker_id = None, transcription=None, translation=None):
        self.start = start
        self.end = end
        self.speaker_id = speaker_id
        self.speaker_dict = speaker_dict
        self.transcription = transcription
        self.translation = translation

    def set_speaker_id(self, speaker_id):
        if speaker_id not in self.speaker_dict.keys():
            self.speaker_dict[speaker_id]= speaker_id
        self.speaker_id = speaker_id
            
    
    def set_end(self, end):
        self.end = end
    
    def get_end(self):
        return self.end
    
    def get_start(self):
        return self.start
    
    def get_transcription(self):
        return self.transcription
    
    def get_translation(self):
        return self.translation
    
    def get_speaker(self):
        return self.speaker_dict[self.speaker_id] if self.speaker_id in self.speaker_dict.keys() else None
    
    def append_translation(self, additional_text):
        self.translation += additional_text

    def append_transcription(self, additional_text):
        self.transcription += additional_text

    def to_df(self):
        return pd.DataFrame({
            'start': [self.start],
            'end': [self.end],
            'speaker': [self.get_speaker()],
            'transcription': [self.transcription],
            'translation': [self.translation],
            })

    def __str__(self):
        start = self.start
        end = self.end
        speaker = self.get_speaker()
        transcription = self.transcription
        translation = self.translation
        string_value = f'[{start} ---> {end}] {speaker}: \n Transcription: {transcription} \n Translation: {translation} \n'
        print(string_value)
        return string_value
    
audio_files = [
            #    "De_Dienst_promo",
               "De_Dienst_ep_1",
               "De_Dienst_ep_2",
               "De_Dienst_ep_3",
               "De_Dienst_ep_4",
               "De_Dienst_ep_5",
               "De_Dienst_ep_6"
]

def insert_interval(interval_list, new_interval, speaker_dict, max_interruption=datetime.timedelta(seconds=1.5)):
    """Check whether a new interval has to be created, of if the current interval should be merged with an existing one.

    Args:
        interval_list (_List_): A list of all speaker intervals
        new_interval (_Dict_): The new interval that is to be inserted into the list
        speaker_dict (_Dict_): A dictionary that maps speaker labels to speaker names
        max_interruption (_datetime.timedelta_, optional): The max duration between intervals. If this time is exceeded, the it will be split into two separate intervals. Defaults to datetime.timedelta(seconds=1.5).
    """
    if speaker_dict[new_interval['label']] != None: # if the speaker is known, we can potentially merge the interval with an existing one
        nearest_intervals = list(filter(
            lambda item: item[1]['end'] > new_interval['start'] - max_interruption, enumerate(interval_list)))
        
        mergeable_interval = list(filter(
            lambda item: item[1]['label'] == new_interval['label'] or speaker_dict[item[1]['label']] == None , nearest_intervals))
        
        # print(mergeable_interval)
        
        if len(mergeable_interval) > 0: # if the length of the mergeable intervals is greater than 0, we can merge the new interval with an existing one
            assert len(mergeable_interval) == 1
            idx, _ = mergeable_interval[0]
            interval_list[idx]['end'] = new_interval['end']
        else: # otherwise, we have to create a new interval
            interval_list.append(new_interval)
    else: # if the speaker is not known, we have to create a new interval
        idx, last_interval = max( enumerate(interval_list), 
            key = lambda item: item[1]['end']
        )
        interval_list[idx]['end'] = last_interval['end']


def refine_speaker_intervals(diarization, audio_file, speaker_dict, min_frag_length = 0.5):
    """The speaker intervals produced by the diarization pipeline are somewhat noisy, with many short segments that are not useful for the transcription. 
    This function plays the audio segments and asks the user to identify the speaker if the speaker is not known yet. It also merges segments that are close together and have the same speaker.

    Args:
        diarization (_type_): The diarization produced by the pyannote pipeline.
        audio_file (_type_): the mp3 file used in the diarization.
        speaker_dict (_type_): The mapping between the indices of the distinct speakers and their names. 
        min_frag_length (float, optional): The minimum length of a speaker interval in seconds. Defaults to 0.5.

    Returns:
        _type_: A list of speaker intervals, where each interval is a dictionary with the keys 'start', 'end', and 'label'.
    """
    speaker_interval_list = []
    audio = AudioSegment.from_mp3(audio_file)

    for seg in diarization.itersegments(): # iterate over all segments in the diarization
        assert len(diarization.get_labels(seg)) == 1 # each segment should have exactly one label
        interval = {
            'start': datetime.timedelta(seconds=seg.start),
            'end': datetime.timedelta(seconds=seg.end),
            'label': diarization.get_labels(seg).pop()
        }
        if speaker_dict[interval['label']] == None and seg.end-seg.start > min_frag_length: # if the speaker is not known and the segment is long enough, we ask the user to identify the speaker
            audio_seg = audio[seg.start*1000:seg.end*1000]
            play(audio_seg)
            name = input('Who is this?').strip() or None
            speaker_dict[interval['label']] = name
        if len(speaker_interval_list) == 0: # if the speaker interval list is still empty, we can just add the interval
            speaker_interval_list.append(interval)
        else: # otherwise, we have to check whether the interval can be merged with an existing one
            insert_interval(speaker_interval_list, interval, speaker_dict)
    return speaker_interval_list

def transcribe_and_translate(audio, language):
    """Function to transcribe and translate audio using the Whisper model. Based on https://github.com/openai/whisper/blob/main/notebooks/Multilingual_ASR.ipynb, commit f0083e7eb20d032390e42f6f6039947fa8669c93
    

    Args:
        audio (_String_): location of the audio file to be transcribed and translated
        language (_String_): Language of the speech contained within the audio file, e.g. 'Dutch' or 'Spanish'.

    Returns:
        (Dict, Dict): Two dictionaries, one for the transcription and one for the translation. Each dictionary contains, among other things, a list of segments.
    """
    model = whisper.load_model("medium", device = 'cuda:0')
    print(
        f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
        f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
    )

    options = dict(language=language, beam_size=5, best_of=5, verbose=False)
    transcribe_options = dict(task="transcribe", **options)
    translate_options = dict(task="translate", **options)

    print("---- starting transcription ----")
    transcription = model.transcribe(audio, **transcribe_options)
    print("---- starting translation ----")
    translation = model.transcribe(audio, **translate_options)
    return transcription, translation

def find_speaker_id(speaker_interval_list, line, audio, error_margin = datetime.timedelta(seconds=2)):
    """The intervals produced by the diarization model and the transcriptions/translations are not always perfectly aligned.
    This function tries to find which speaker identified by the diarization model is speaker at the time of the transcription/translation line.

    Args:
        speaker_interval_list (_type_): The speaker intervals as identified by the diarization model.
        line (_type_): The current transcription line for which we want to find the speaker.
        audio (_type_): the audio file used in the diarization.
        error_margin (_type_, optional): The beginning and end points of the transcription and translation lines are do not always match with those of the speaker interval list. Some error margin is therefore needed.  Defaults to datetime.timedelta(seconds=2).

    Returns:
        _type_: _description_
    """
    start = line.get_start()
    end = line.get_end()
    transcript = line.get_transcription()
    overlapping_intervals = list(filter(
        lambda interval: (interval['start'] < start + error_margin and interval['end'] > end - error_margin), speaker_interval_list))
    if len(overlapping_intervals) == 0: # if there are no overlapping intervals, we cannot find a speaker automatically so the user has to identify the speaker manually
        print(f'Segment [{start} ---> {end}] could not be matched with a single speaker \n Transcript: {transcript}')
        audio_seg = audio[start.total_seconds()*1000:end.total_seconds()*1000]
        play(audio_seg)
        name = input('Could not identify the speaker. Please enter manually').strip() or None
        return name if name != None else find_speaker_id(speaker_interval_list, line, audio, error_margin)
    elif len(overlapping_intervals) == 1: # if there is only one overlapping interval, we can return the label of that interval
        return overlapping_intervals[0]['label']
    else: # if there are multiple overlapping intervals, we return the shortest one
        shortest_interval = min(overlapping_intervals, key=lambda interval: interval['end'] - interval['start'])
        return shortest_interval['label']

def merge_transcription_and_translation(transcription, translation, speaker_interval_list, audio_file, speaker_dict):
    """Unfortunately, the transcription and translation segments do not always match perfectly.
    This function tries to merge the two segments into one list of TranscriptLine objects, which contain both the transcription and translation text, as well as the speaker information.

    Args:
        transcription (_type_): The transcription produced by the Whisper model.
        translation (_type_): The translation produced by the Whisper model.
        speaker_interval_list (_type_): The speaker intervals as identified by the diarization model, and cleaned up by the refine_speaker_intervals function.
        audio_file (_type_): The destination of used the audio file.
        speaker_dict (_type_): A dictionary matching the indices in the speaker_interval_list to the speaker names.

    Raises:
        Exception: If somehow during the previous iteration of the while loop no 'transcription', translation' or 'both' lines were popped, variables are not set correctly, which should not be possible.

    Returns:
        _type_: _description_
    """
    transcr_iter = iter(transcription['segments'])
    transl_iter = iter(translation['segments'])
    transcr_line = next(transcr_iter)
    transl_line = next(transl_iter)
    
    audio = AudioSegment.from_mp3(audio_file)
    lines = []

    prev_pop = None

    while transcr_line != None and transl_line != None: # Iterate over both transcription and translation segments until both are empty
        if transcr_line == None or transl_line == None: # If one of the segments is empty, something is wrong and we should investigate
            print(f'help! transcr: {transcr_line}, transl:{transl_line}')
        if prev_pop == 'both' or prev_pop == None: # If during the previous iteration both a transcript and a translation line is popped, we create a new TranscriptLine object
            if abs(transcr_line['start']-transl_line['start']) > 1: # if the starting times of the transcription and translation do not match, we print a warning
                print(f'Starting times are not matching. Check the following lines ---- \n Transcription: {transcr_line["text"]} \n Translation: {transl_line["text"]}')
            line = TranscriptLine(start=datetime.timedelta(seconds=transcr_line['start']), 
                                transcription=transcr_line['text'], 
                                translation=transl_line['text'],
                                speaker_dict=speaker_dict)
        elif prev_pop == 'transcr': # If during the previous iteration only a transcription line was popped, we append the translation text to the current TranscriptLine object
            line.append_transcription(transcr_line['text'])
        elif prev_pop == 'transl': # If during the previous iteration only a translation line was popped, we append the transcription text to the current TranscriptLine object
            line.append_translation(transl_line['text'])
        else:
            raise Exception('This should not be possible')
        
        if abs(transcr_line['end']-transl_line['end']) <= 1: # If the ending times of the transcription and translation match sufficiently, we can set the end time of the TranscriptLine object and pop both a transcript line and a translation line
            line.set_end(datetime.timedelta(seconds=transcr_line['end']))
            speaker_id = find_speaker_id(speaker_interval_list, line, audio)
            line.set_speaker_id(speaker_id)
            lines.append(line)
            try:
                transcr_line = next(transcr_iter)
                transl_line = next(transl_iter)
                prev_pop = 'both'
            except StopIteration:
                break
        elif transcr_line['end'] < transl_line['end']: # If the end time of the transcription line is before the end time of the translation line, we pop another transcription line
            try:
                transcr_line = next(transcr_iter)
                prev_pop = 'transcr'
            except StopIteration:
                break
        else: # If the end time of the translation line is before the end time of the transcription line, we pop another translation line
            try:
                transl_line = next(transl_iter)
                prev_pop = 'transl'
            except StopIteration:
                break
    return lines

def concat_speaker_segments(lines):
    """If the time between two speaker segments with the same speaker is less than 2 seconds, we merge the two segments into one.

    Args:
        lines (List): the speaker segments that are to be concatenated, produced by the merge_transcription_and_translation function.

    Returns:
        panda.Dataframe: A dataframe containing the concatenated speaker segments.
    """
    segments = []
    segment = lines[0].to_df()

    for l in lines[1:]:
        print(segment)
        if segment.at[0, 'speaker'] == l.get_speaker() and (segment.at[0, 'end'] - l.get_start()) < datetime.timedelta(seconds=2):
            print(f'merging segment:')
            print(f'{segment}')
            print('with:')
            print(f'{l.to_df()}')
            print(' ')
            segment.at[0, 'transcription'] = segment.at[0, 'transcription'] + l.get_transcription()
            segment.at[0, 'translation'] = segment.at[0, 'translation'] + l.get_translation()
            segment.at[0, 'end'] = l.get_end()
        else:
            segments.append(segment)
            segment = l.to_df()
            

    segments.append(segment)
    return pd.concat(segments)

def add_seg_id(df, file_name):
    """Function to add a segment ID to each row of the dataframe. 

    Args:
        df (pandas.DataFrame): The dataframe to which the segment ID is to be added.
        file_name (String): The name of the audio file, which is used to create the segment ID.

    Returns:
        pandas.DataFrame: A dataframe with an additional column 'seg_id', which contains the segment ID for each row.q
    """
    ids = [file_name+'_seg_'+str(i) for i in range(df.shape[0])]
    df.loc[:, 'seg_id'] = ids
    col_names = df.columns.tolist()
    new_order = col_names[-1:] + col_names[:-1] 
    df = df[new_order]
    return df


def change_time_to_abs_values(df, ep_nr):
    """In order to preserve the chronological order of the speaker intervals across multiple episodes, we add a full day to the start and end times with each consecutive episode.

    Args:
        df (pandas.dataframe): The dataframe containing the speaker intervals.
        ep_nr (Integer): the number of the episode that is being processed. 

    Returns:
        pandas.Dataframe: A dataframe with speaker intervals where the start and and times are updated. 
    """
    if ep_nr == 'promo':
        start_time = datetime.date(2024, 12, 31)
    else:
        start_time = datetime.date(2025, 1, int(ep_nr))
    print(df['start'])
    df['start'] = df['start'].apply(lambda x:  datetime.datetime.combine(start_time, (datetime.datetime.min + x).time()))
    df['end'] = df['end'].apply(lambda x: datetime.datetime.combine(start_time, (datetime.datetime.min + x).time()))
    return df

            

def print_df(df, file):
    """Function to print a dataframe to a csv file. The file is saved in the transcriptions/dataframes directory.

    Args:
        df (pandas.Dataframe): The dataframe to be printed.
        file (String): The name of the file to which the dataframe is to be printed, without the .csv extension.
    """
    full_path = "./transcriptions/dataframes/" + file + '.csv'
    df.to_csv(full_path, sep=';', index=False, header=True)


def print_lines(lines, file):
    """Function to print a list of TranscriptLine objects to a text file. The file is saved in the transcriptions/text directory.

    Args:
        lines (List): A list of TranscriptLine objects to be printed.
        file (String): The name of the file to which the lines are to be printed, without the .txt extension.
    """
    full_path = "./transcriptions/text/" + file + '.txt'
    with open(full_path, "w") as text_file:
        for line in lines:
            text_file.write(str(line))

def print_convo(convo, file):
    """Function to print a dataframe is the form of a 'conversation', meaning that each row contains the start and end time, the speaker, the transcription and the translation of a segment. The file is saved in the transcriptions/text directory.

    Args:
        convo (_type_): A dataframe containing the conversation to be printed. The dataframe should contain the columns 'start', 'end', 'speaker', 'transcription' and 'translation'.
        file (_type_): The name of the file to which the conversation is to be printed, without the .txt extension.
    """
    full_path = "./transcriptions/text/" + file + '.txt'
    with open(full_path, "w") as text_file:
        for idx, row in convo.iterrows():
            text = f'[{row["start"]} ---> {row["end"]}] {row["speaker"]}: \n Transcription: {row["transcription"]} \n Translation: {row["translation"]} \n'
            text_file.write(text)   

# Load secrets from a yaml file
secrets = yaml.load(open('./secrets.yaml'), Loader=yaml.FullLoader)      



In [32]:
# file = audio_files[0]
for file in audio_files:
    print(f"processing {file}")

    audio_file = f"./audio_files/uncut/{file}.mp3"
    # instantiate the pipeline for diarizing audio fragment
    pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization-3.1",
    use_auth_token=secrets['hugging_face']['use_auth_token'])
    pipeline = pipeline.to(torch.device('cuda:0'))

    # run the pipeline on an audio file
    max_speakers = 5

    print('---- diarizing audio fragment ----')
    diarization = pipeline(audio_file, min_speakers=1, max_speakers=max_speakers)

    speaker_dict = {
    f'SPEAKER_0{n}':None for n in range(max_speakers)
    }
    speaker_dict['UNKNOWN'] = 'unknown'

    print('---- turning fragments into speaker intervals ----')
    speaker_interval_list = refine_speaker_intervals(diarization, audio_file, speaker_dict)


    language = "Dutch"

    print('---- transcribing and translating audio fragment ----')
    transcription, translation = transcribe_and_translate(audio_file, language)

    print('---- merging everything ----')
    lines = merge_transcription_and_translation(transcription, translation, speaker_interval_list, audio_file, speaker_dict)

    print('---- concatenating speaker segments')
    convo  = concat_speaker_segments(lines)

    ep_nr = file.split('_')[-1] 
    convo = change_time_to_abs_values(convo, ep_nr)
    convo = add_seg_id(convo, file)

    print('---- exporting results ----')

    print_df(convo, file)
    print_convo(convo, file)



# with open("./transcriptions/"+file+".txt", "w") as text_file:
#     for line in result['segments']:
#         text_file.write(str(line['text']))
#         text_file.write('\n')

processing De_Dienst_ep_1
---- diarizing audio fragment ----
---- turning fragments into speaker intervals ----


Input #0, wav, from '/tmp/tmp36m9sql5.wav':   0KB sq=    0B f=0/0   
  Duration: 00:00:08.66, bitrate: 1411 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 44100 Hz, 2 channels, s16, 1411 kb/s


Input #0, wav, from '/tmp/tmpet3txhfz.wav':   0KB sq=    0B f=0/0   
  Duration: 00:00:05.67, bitrate: 1411 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 44100 Hz, 2 channels, s16, 1411 kb/s


Input #0, wav, from '/tmp/tmpvw8sf7qg.wav':   0KB sq=    0B f=0/0   
  Duration: 00:00:00.68, bitrate: 1411 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 44100 Hz, 2 channels, s16, 1411 kb/s


Input #0, wav, from '/tmp/tmp1ysv9lqz.wav':   0KB sq=    0B f=0/0   
  Duration: 00:00:00.86, bitrate: 1411 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 44100 Hz, 2 channels, s16, 1411 kb/s


...


Starting times are not matching. Check the following lines ---- 
 Transcription:  Ja, de casus is klaar. 
 Translation:  Well, that seems nice.
Segment [0:47:35 ---> 0:47:45] could not be matched with a single speaker 
 Transcript:  Mijn zesdaagse AIVD-carrière zit erop.


Input #0, wav, from '/tmp/tmpafigkuw6.wav':   0KB sq=    0B f=0/0   
  Duration: 00:00:10.00, bitrate: 1411 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 44100 Hz, 2 channels, s16, 1411 kb/s



Starting times are not matching. Check the following lines ---- 
 Transcription:  naar huis. 
 Translation:  This was the last episode
Segment [0:47:50 ---> 0:47:55] could not be matched with a single speaker 
 Transcript:  naar huis. Dit was de laatste aflevering van De Dienst.


Input #0, wav, from '/tmp/tmpv_j_qj8u.wav':   0KB sq=    0B f=0/0   
  Duration: 00:00:05.00, bitrate: 1411 kb/s
  Stream #0:0: Audio: pcm_s16le ([1][0][0][0] / 0x0001), 44100 Hz, 2 channels, s16, 1411 kb/s



---- concatenating speaker segments
   start             end speaker  \
0 0 days 0 days 00:00:04    Bart   

                                       transcription  \
0   Hoi Elisabeth. Ja, je vraagt je natuurlijk af...   

                                         translation  
0   Hi Elisabeth. Yes, of course you are wonderin...  
merging segment:
   start             end speaker  \
0 0 days 0 days 00:00:04    Bart   

                                       transcription  \
0   Hoi Elisabeth. Ja, je vraagt je natuurlijk af...   

                                         translation  
0   Hi Elisabeth. Yes, of course you are wonderin...  
with:
            start             end speaker  \
0 0 days 00:00:04 0 days 00:00:06    Bart   

                            transcription                    translation  
0   Ja, dat kan niet veel goed betekenen.   Yes, that doesn't mean much.  
 
   start             end speaker  \
0 0 days 0 days 00:00:06    Bart   

                                